In [8]:
! pip install jupyter notebook jupyter_contrib_nbextensions

  Running setup.py bdist_wheel for jupyter-nbextensions-configurator: started
  Running setup.py bdist_wheel for jupyter-nbextensions-configurator: finished with status 'done'
  Stored in directory: C:\Users\salee\AppData\Local\pip\Cache\wheels\15\df\fe\2a74fe34709e7fdc5ae153a768675d9fda93cc7d5133ed1fb0
  Running setup.py bdist_wheel for jupyter-latex-envs: started
  Running setup.py bdist_wheel for jupyter-latex-envs: finished with status 'done'
  Stored in directory: C:\Users\salee\AppData\Local\pip\Cache\wheels\0d\71\2a\164491997299b9f2479a251e254323fe35d946779e18f27956
Successfully built jupyter-nbextensions-configurator jupyter-latex-envs


In [12]:
!pip install https://download.pytorch.org/whl/cpu/torch-1.0.1-cp37-cp37m-win_amd64.whl
!pip install torchvision

  Using cached https://files.pythonhosted.org/packages/fb/01/03fd7e503c16b3dc262483e5555ad40974ab5da8b9879e164b56c1f4ef6f/torchvision-0.2.2.post3-py2.py3-none-any.whl


In [5]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

from fastai.imports import *
from fastai.tabular import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
!pip install pandas-summary

  Using cached https://files.pythonhosted.org/packages/6b/00/f7b4d7fd901db9a79d63e88000bd1e12efba4f5fb52608f906d7fea2b18f/pandas_summary-0.0.6-py2.py3-none-any.whl


In [6]:
import pandas as pd

df_raw = pd.read_csv('Train.csv', low_memory=False, 
                     parse_dates=["saledate"])